In [1]:
refresh_embeddings = False

In [13]:
import os
os.chdir('/workspaces/llm-dungeon-master')

In [14]:
import glob
from langchain_community.document_loaders import TextLoader

import sys
sys.path.append('~/workspaces/llm-dungeon-master')

sheet_files = glob.glob('benchmarks/character-sheets/*.txt')
character_sheets = []

for sheet in sheet_files:
    loader = TextLoader(sheet)
    character_sheets += loader.load()


In [17]:
from langchain_community.vectorstores import LanceDB
from langchain.embeddings.openai import OpenAIEmbeddings
import lancedb

if refresh_embeddings:

    embeddings = OpenAIEmbeddings()

    db = lancedb.connect("/workspaces/llm-dungeon-master/db")
    table = db.create_table(
        "character_sheets",
        data=[
            {
                "vector": embeddings.embed_query("Hello World"),
                "text": "Hello World",
                "id": "1",
            }
        ],
        mode="overwrite",
    )

    # Load the document, split it into chunks, embed each chunk and load it into the vector store.
    vectorstore = LanceDB.from_documents(character_sheets, OpenAIEmbeddings(), connection=table)

In [18]:
query = "What is Aros's charisma?"
docs = vectorstore.similarity_search(query)
print(docs[0:1])

[Document(page_content='Name: Aros the Unforgiving\nRace/Ancestry/Heritage: Fallen Aasimar\nClass & Level: Cleric 1\nBackground: Spy\nAlignment: Chaotic Good\nExperience: 0 xp\n\n\nABILITY SCORES & ABILITIES\n(* includes +2 proficiency bonus; ** includes expertise, if any)\n\nStrength 14\n+2 ... Strength Ability Checks\n+2 ... Strength Saving Throws\n+2 ... Athletics Skill\n210 lbs. ... Maximum Carrying Capacity\n420 lbs. ... Max. Push or Drag (Speed -5 ft.)\n420 lbs. ... Maximum Lift\n\nDexterity 8\n-1 ... Dexterity Ability Checks\n-1 ... Dexterity Saving Throws\n-1 ... Acrobatics Skill\n-1 ... Sleight of Hand Skill\n+1 ... Stealth Skill  * (Disadv.)\n\n\nConstitution 14\n+2 ... Constitution Ability Checks\n+2 ... Constitution Saving Throws\n\nIntelligence 10\n+0 ... Intelligence Ability Checks\n+0 ... Intelligence Saving Throws\n+0 ... Arcana Skill\n+0 ... History Skill\n+0 ... Investigation Skill\n+0 ... Nature Skill\n+2 ... Religion Skill  *\n\nWisdom 15\n+2 ... Wisdom Ability Chec

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

if not refresh_embeddings:
    embedding_function = OpenAIEmbeddings()


    db = lancedb.connect('db')
    table = db.open_table('character_sheets')
    vectorstore = LanceDB(table, embedding_function)

# Create a retriever that fetches documents from multiple tables
lance_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

docs = lance_retriever.get_relevant_documents("What is Aros's charisma?")
print(docs)
template = """Answer the question based only on the following context:
{context}.

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": lance_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

[Document(page_content='Name: Aros the Unforgiving\nRace/Ancestry/Heritage: Fallen Aasimar\nClass & Level: Cleric 1\nBackground: Spy\nAlignment: Chaotic Good\nExperience: 0 xp\n\n\nABILITY SCORES & ABILITIES\n(* includes +2 proficiency bonus; ** includes expertise, if any)\n\nStrength 14\n+2 ... Strength Ability Checks\n+2 ... Strength Saving Throws\n+2 ... Athletics Skill\n210 lbs. ... Maximum Carrying Capacity\n420 lbs. ... Max. Push or Drag (Speed -5 ft.)\n420 lbs. ... Maximum Lift\n\nDexterity 8\n-1 ... Dexterity Ability Checks\n-1 ... Dexterity Saving Throws\n-1 ... Acrobatics Skill\n-1 ... Sleight of Hand Skill\n+1 ... Stealth Skill  * (Disadv.)\n\n\nConstitution 14\n+2 ... Constitution Ability Checks\n+2 ... Constitution Saving Throws\n\nIntelligence 10\n+0 ... Intelligence Ability Checks\n+0 ... Intelligence Saving Throws\n+0 ... Arcana Skill\n+0 ... History Skill\n+0 ... Investigation Skill\n+0 ... Nature Skill\n+2 ... Religion Skill  *\n\nWisdom 15\n+2 ... Wisdom Ability Chec

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [20]:
chain.invoke("What are Gualluwyrm's weapons?")

"Gualluwyrm's weapons are a shortsword, a longbow, and 20 arrows."

In [ ]:
import os

def get_folder_size(folder):
    total = 0
    for path, dirs, files in os.walk(folder):
        for f in files:
            fp = os.path.join(path, f)
            total += os.path.getsize(fp)
    return total

# Usage
folder_size = get_folder_size("/workspaces/llm-dungeon-master/db")
print(f"The size of the folder is {folder_size} bytes.")

The size of the folder is 16669991 bytes.
